# Assignment 4

Deadline: 30.04.2025 12:00 CET

<Add your name, student-id and emal address>

## Prerequisites: Library imports, data load and initialization of the backtest service

In [3]:
# Standard library imports
import os
import sys
import copy
from typing import Optional

# Third party imports
import numpy as np
import pandas as pd

# Add the project root directory to Python path
#project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
#src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
os.chdir("C:/Users/angus/OneDrive/Desktop/QPMwP/Investment_Strategy")
project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')

sys.path.append(project_root)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.constraints import Constraints
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    bibfn_selection_min_volume,
    bibfn_selection_gaps,
    bibfn_return_series,
    bibfn_budget_constraint,
    bibfn_box_constraints,
    bibfn_size_dependent_upper_bounds,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest

In [6]:
# Load data
path_to_data = "C:/Users/angus/OneDrive/Desktop/QPMwP/Investment_Strategy/data/"  # <change this to your path to data>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}market_data.parquet')

# Instantiate the BacktestData class
# and set the market data and jkp data as attributes
data = BacktestData()
data.market_data = market_data
data.bm_series = load_data_spi(path="C:/Users/angus/OneDrive/Desktop/QPMwP/Investment_Strategy/data/")  # <change this if necessary>

In [7]:
# Define rebalancing dates
n_days = 21*3
market_data_dates = market_data.index.get_level_values('date').unique().sort_values(ascending=True)
rebdates = market_data_dates[market_data_dates > '2015-01-01'][::n_days].strftime('%Y-%m-%d').tolist()

In [8]:
# Define the selection item builders.
selection_item_builders = {
    'gaps': SelectionItemBuilder(
        bibfn = bibfn_selection_gaps,
        width = 252*3,
        n_days = 10,
    ),
    'min_volume': SelectionItemBuilder(
        bibfn = bibfn_selection_min_volume,
        width = 252,
        min_volume = 500_000,
        agg_fn = np.median,
    ),
}

# Define the optimization item builders.
optimization_item_builders = {
    'return_series': OptimizationItemBuilder(
        bibfn = bibfn_return_series,
        width = 252*3,
        fill_value = 0,
    ),
    'budget_constraint': OptimizationItemBuilder(
        bibfn = bibfn_budget_constraint,
        budget = 1,
    ),
    'box_constraints': OptimizationItemBuilder(
        bibfn = bibfn_box_constraints,
        upper = 0.1,
    ),
    'size_dep_upper_bounds': OptimizationItemBuilder(
        bibfn = bibfn_size_dependent_upper_bounds,
        small_cap = {'threshold': 300_000_000, 'upper': 0.02},
        mid_cap = {'threshold': 1_000_000_000, 'upper': 0.05},
        large_cap = {'threshold': 10_000_000_000, 'upper': 0.1},
    ),
}

# Initialize the backtest service
bs = BacktestService(
    data = data,
    selection_item_builders = selection_item_builders,
    optimization_item_builders = optimization_item_builders,
    rebdates = rebdates,
)

## 1. Maximum Sharpe Ratio Portfolio

a) 

(6 points)

Complete the `MaxSharpe` class below by implementing your its methods `set_objective` and `solve`.
The `solve` method should implement an iterative algorithm that quickly approximates the "true" maximimum Sharpe ratio portfolio (given the estimates of mean and covariance). This approximation should be done by repeatedly solving a mean-variance optimization problem, where the risk aversion parameter (which scales the covariance matrix) is adjusted in each iteration. The algorithm should terminate after a maximum of 10 iterations. 

In [9]:
class MaxSharpe(Optimization):

    def __init__(self,
                 constraints: Optional[Constraints] = None,
                 covariance: Optional[Covariance] = None,
                 expected_return: Optional[ExpectedReturn] = None,
                 **kwargs) -> None:
        super().__init__(
            constraints=constraints,
            **kwargs,
        )
        self.covariance = Covariance() if covariance is None else covariance
        self.expected_return = ExpectedReturn() if expected_return is None else expected_return


    def set_objective(self, optimization_data: OptimizationData) -> None:
        self.data = optimization_data
        X = optimization_data["return_series"]
        self.mu  = self.expected_return.estimate(X=X, inplace=False)
        self.cov = self.covariance.estimate(X=X, inplace=False)
        return None

    def solve(self) -> None:
        def _evaluate_lambda(lam: float) -> tuple[pd.Series, float]:
            meanvar = MeanVariance(
                constraints=self.constraints,
                covariance=self.covariance,
                expected_return=self.expected_return,
                risk_aversion=lam,
                solver_name="cvxopt",
            )
            meanvar.set_objective(self.data)
            meanvar.solve()
            weight = pd.Series(meanvar.results["weights"])

           

            sharpe = float((self.mu @ weight) / np.sqrt(weight @ self.cov @ weight))
            return weight, sharpe
                                           
        left, right = 1e-2, 1e2      
        iterations = 10                 

        best_sharpe = -np.inf
        best_weight = None
        best_lambda = None

        for i in range(iterations):
            # golden-section interior points
            lam1 = left + 0.382 * (right - left)
            lam2 = left + 0.618 * (right - left)

            weight1, sharpe1 = _evaluate_lambda(lam1)
            weight2, sharpe2 = _evaluate_lambda(lam2)

            # keep global best so far
            if sharpe1 > best_sharpe:
                best_sharpe, best_weight, best_lambda = sharpe1, weight1, lam1
            if sharpe2 > best_sharpe:
                best_sharpe, best_weight, best_lambda = sharpe2, weight2, lam2

            # shrink bracket based on unimodality of SR(λ)
            if sharpe1 < sharpe2:
                left = lam1          
            else:
                right = lam2        

        self.results = {
            "weights": best_weight.to_dict(),
            "best_sharpe": best_sharpe,
            "risk_aversion": best_lambda,
            "status": True,
        }
        
        return None

b) 

(2 points)

Provide a theoretical or empirical justification that your algorithm converges to the true maximum Sharpe ratio portfolio for the given coefficients of mean and covariance.
Hint: If you want to provide an empirical justification, you can perform an optimization for a single point in time by running the following code.

In [10]:
bs.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',  # <change this to your preferred solver>
    #<add any other parameters you need, e.g., number of iterations, tolerance, etc.>
)
bs.prepare_rebalancing('2015-01-02')
bs.optimization.set_objective(bs.optimization_data)
bs.optimization.solve()
bs.optimization.results

Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson


{'weights': {'102': 2.3931874043434964e-07,
  '103': 1.588866979672604e-07,
  '104': 0.019999168903181232,
  '111': 0.018575718640264815,
  '113': 5.064408182352598e-08,
  '122': 4.117078788107552e-07,
  '125': 0.019999759102146755,
  '127': 0.01999971625116084,
  '128': 0.019999604842537974,
  '13': 3.259181946362215e-07,
  '139': 0.019999636812474506,
  '141': 0.01999979688549057,
  '142': 2.154602599595474e-07,
  '148': 0.019999771376754377,
  '149': 2.09555928327806e-07,
  '154': 3.6150053338311065e-07,
  '159': 0.01999931556560766,
  '161': 0.006379762104453635,
  '169': 0.0199997889585858,
  '175': 0.019999053274222932,
  '176': 0.019996819242999047,
  '177': 1.7525292370921126e-07,
  '184': 0.019999447887238394,
  '185': 0.019999208398926306,
  '187': 2.9876140835635394e-07,
  '189': 7.895114239299329e-07,
  '191': 2.184727563952155e-06,
  '197': 8.576063895563821e-07,
  '2': 0.019997596373033888,
  '200': 0.01999756921260904,
  '201': 0.019999053723481413,
  '204': 0.0199998005

## 2. Backtest MaxSharpe with Turnover Penalty

(5 points)

Calibrate the turnover penalty parameter such that the backtest of the MaxSharpe strategy displays an annual turnover of roughly 100%.

In [11]:
# Update the backtest service with a MaxSharpe optimization object
penalties  = [ 1e-3, 5e-3, 1e-2, 5e-2, 0.1, 0.5, 1, 5, 10]
closest_penalty = None
closest_gap= 100

for i in list(penalties):
    #bs_copy = copy.deepcopy(bs)
    #bs_copy.optimization = 
    bs.optimization = MaxSharpe(
    covariance = Covariance(method='pearson'),
    expected_return = ExpectedReturn(method='geometric'),
    solver_name = 'cvxopt',    # <change this to your preferred solver>
    turnover_penalty= i,   # <change this>
    )

    backtest_trials = Backtest()
    #backtest_trials.run(bs=bs_copy)
    backtest_trials.run(bs=bs)
    
    #annual_turnover = backtest_trials.strategy.turnover(return_series = bs_copy.data.get_return_series()).mean() * 4
    annual_turnover = backtest_trials.strategy.turnover(return_series = bs.data.get_return_series()).mean() * 4
    tolerance = 0.05
    gap = abs(annual_turnover - 1.0)
    if gap < closest_gap:
        if gap < tolerance:
            closest_penalty = i
            break
        else:
            closest_gap = gap
            closest_penalty = i
            new_point1 = i + i/2
            newpoint2 = i - i/2
            if new_point1 not in penalties:
                penalties.extend([new_point1])
            if newpoint2 not in penalties:
                penalties.extend([newpoint2])
    
    print(gap)
    print(closest_gap)
    print(i)

print(closest_penalty)

bs.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',    # <change this to your preferred solver>
    turnover_penalty=closest_penalty,   # <change this>
)

# Instantiate the backtest object
bt_ms = Backtest()

# Run the backtest
bt_ms.run(bs = bs)

Rebalancing date: 2015-01-02
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Rebalancing date: 2015-04-01
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: pearson
Covariance estimation method: 

KeyboardInterrupt: 

## 3. Simulation and Descriptive Statistics

(3 points)

- Simulate the portfolio returns from your MaxSharpe backtest. Use fixed costs of 1% and variable costs of 0.3%.
- Plot the cumulated returns of the MaxSharpe strategy together with those of the SPI Index.
- Plot the turnover of your MaxSharpe strategy over time.
- Print the annualized turnover (computed as the average turnover over the backtest multiplied by the number of rebalancing per year) for your MaxSharpe strategy.
- Create and print a table with descriptive performance statistics for your MaxSharpe strategy and the SPI Index.


In [ ]:
#<your code here>